Results
- Number of events
- Min date, max date
- Ratio true false
- Number of unique users
- Number of unique sessions
- Matrix of eval users and sessions in train
- Matrix of test users and sessions in eval
- Matrix of test users and sessions in train


In [3]:
import pandas as pd
import json
import os

In [4]:
import cudf

In [5]:
CORE_PATH = 'calculated_features'

In [11]:
!ls ../../../datasets/calculated_features

289ed5d9-0a4e-4f14-8000-5f20eabc10d8.ics
PGPMessage.pdf
basic_descriptive_30_files.json
core_dict.json
files_used_2.parquet


In [10]:
path = os.path.join(CORE_PATH, 'files_used_10')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [7]:
df = cudf.read_parquet(path)

In [12]:
description = df.describe().round(4)

In [14]:
description.columns

Index(['index', 'project_id', 'session_5_count', 'session_30_count', 'user_id',
       'country_count', 'date_hour', 'date_minute', 'date_hour_sin',
       'date_hour_cos', 'date_minute_sin', 'date_minute_cos',
       'cum_session_event_count', 'delta_last_event', 'cum_session_time',
       'cum_platform_time', 'cum_platform_events', 'cum_projects',
       'average_event_time', 'expanding_click_average', 'rolling_session_time',
       'rolling_session_events', 'rolling_session_gap', 'session_event_count',
       'session_time_minutes', 'previous_session_time',
       'previous_session_events', 'session_30_raw', 'global_session_time',
       'global_events_user', 'year', 'month', 'day', 'hour', 'minute',
       'second', 'cum_session_time_raw', 'cum_platform_time_raw',
       'cum_session_event_raw', 'cum_platform_event_raw',
       'session_30_count_raw'],
      dtype='object')

In [19]:
description[['delta_last_event', 'expanding_click_average', 'session_30_count']]

,delta_last_event,expanding_click_average,session_30_count
count,12833662.000,12833662.000,12833662.000
mean,29.066,26.518,27.699
std,100.069,54.729,49.603
min,0.000,0.000,1.000
25%,3.000,3.900,2.000
50%,6.000,8.800,8.000
75%,17.000,23.900,30.000
max,1799.000,1316.200,458.000


In [13]:
description.to_csv('calculated_features/files_used_10_description.csv', index=False)

In [12]:
CORE_DICT = {
    'n_events': {
        'train': None,
        'eval': None,
        'test': None
    },
    'dates': {
        'train': None,
        'eval': None,
        'test': None,
    },
    'users': {
        'train': None,
        'eval': None,
        'test': None
    },
    'sessions': {
        'train': None,
        'eval': None,
        'test': None
    },
    'label_count': {
        'train': None,
        'eval': None,
        'test': None  
    },
    'user_crossover': {
        'train_eval': None,
        'train_test': None,
        'eval_test': None
    }
}

In [13]:
df = pd.read_parquet(CORE_PATH, columns=['user_id', 'session_30_raw', 'date_time', 'session_terminates_30_minutes'])
df['label'] = df['session_terminates_30_minutes'].apply(lambda x: not x)
df = df.drop(columns=['session_terminates_30_minutes'])


ArrowInvalid: Error creating dataset. Could not read schema from '../../../datasets/calculated_features/289ed5d9-0a4e-4f14-8000-5f20eabc10d8.ics': Could not open Parquet input source '../../../datasets/calculated_features/289ed5d9-0a4e-4f14-8000-5f20eabc10d8.ics': Parquet magic bytes not found in footer. Either the file is corrupted or this is not a parquet file.. Is this a 'parquet' file?

In [ ]:
# find average click time per user
# find variance of click time per user



In [ ]:
df.head()

,user_id,session_30_raw,date_time,label
0,0,1,2021-10-19 08:40:37,False
1,1,1,2021-10-19 08:40:38,False
2,0,1,2021-10-19 08:40:39,False
3,2,1,2021-10-19 08:40:39,True
4,0,1,2021-10-19 08:40:41,False


In [ ]:
n_rows = df.shape[0]

train_part = (0, int(n_rows * 0.7))
eval_part = (int(n_rows * 0.7), int(n_rows * 0.85))
test_part = (int(n_rows * 0.85), n_rows)

In [ ]:
train, test, eval = df[train_part[0]:train_part[1]], df[test_part[0]:test_part[1]], df[eval_part[0]:eval_part[1]]

In [ ]:
train_label, eval_label, test_label = (
    train['label'].sum() / train.shape[0],
    eval['label'].sum() / eval.shape[0],
    test['label'].sum() / test.shape[0]
)

In [ ]:
train_user_sess, eval_user_sess, test_user_sess = (
    train[['user_id', 'session_30_raw']].drop_duplicates(),
    eval[['user_id', 'session_30_raw']].drop_duplicates(),
    test[['user_id', 'session_30_raw']].drop_duplicates()
)

In [ ]:
core_dict = CORE_DICT.copy()
core_dict['n_events']['train'] = train.shape[0]
core_dict['n_events']['eval'] = eval.shape[0]
core_dict['n_events']['test'] = test.shape[0]

In [ ]:
core_dict['label_count']['train'] = train_label
core_dict['label_count']['eval'] = eval_label
core_dict['label_count']['test'] = test_label

In [ ]:
train_min_max = {'min': str(train['date_time'].min()), 'max': str(train['date_time'].max()) }

eval_min_max = {'min': str(eval['date_time'].min()), 'max': str(eval['date_time'].max()) }
test_min_max = {'min': str(test['date_time'].min()), 'max': str(test['date_time'].max()) }

In [ ]:
core_dict['dates']['train'] = train_min_max
core_dict['dates']['eval'] = eval_min_max
core_dict['dates']['test'] = test_min_max

In [ ]:
core_dict['users']['train'] = train_user_sess['user_id'].unique().shape[0]
core_dict['users']['eval'] = eval_user_sess['user_id'].unique().shape[0]
core_dict['users']['test'] = test_user_sess['user_id'].unique().shape[0]

In [ ]:
core_dict['sessions']['train'] = train_user_sess.shape[0]
core_dict['sessions']['eval'] = eval_user_sess.shape[0]
core_dict['sessions']['test'] = test_user_sess.shape[0]

In [ ]:
train_in_eval = train_user_sess[['user_id']].drop_duplicates().merge(eval_user_sess[['user_id']].drop_duplicates(), how='inner', on='user_id')
train_in_test = train_user_sess[['user_id']].drop_duplicates().merge(test_user_sess[['user_id']].drop_duplicates(), how='inner', on='user_id')
eval_in_test = eval_user_sess[['user_id']].drop_duplicates().merge(test_user_sess[['user_id']].drop_duplicates(), how='inner', on='user_id')
core_dict['user_crossover']['train_eval'] = train_in_eval.shape[0]
core_dict['user_crossover']['train_test'] = train_in_test.shape[0]
core_dict['user_crossover']['eval_test'] = eval_in_test.shape[0]

In [ ]:
train_session_eval = train_user_sess[['user_id', 'session_30_raw']].merge(eval_user_sess[['user_id', 'session_30_raw']], how='inner', on=['user_id', 'session_30_raw'])
eval_session_test = eval_user_sess[['user_id', 'session_30_raw']].merge(test_user_sess[['user_id', 'session_30_raw']], how='inner', on=['user_id', 'session_30_raw'])
core_dict['session_crossover'] = {
    'train_eval': train_session_eval.shape[0],
    'eval_test': eval_session_test.shape[0]
}

In [ ]:
core_dict

{'n_events': {'train': 26950693, 'eval': 5775148, 'test': 5775149},
 'dates': {'train': {'min': '2021-10-19T08:40:37.000000000',
   'max': '2022-06-02T14:42:17.000000000'},
  'eval': {'min': '2022-06-02T14:42:18.000000000',
   'max': '2022-07-09T05:43:02.000000000'},
  'test': {'min': '2022-07-09T05:43:04.000000000',
   'max': '2022-08-14T05:13:27.000000000'}},
 'users': {'train': 81756, 'eval': 20111, 'test': 17376},
 'sessions': {'train': 466103, 'eval': 94549, 'test': 89998},
 'label_count': {'train': 0.38163935153726847,
  'eval': 0.4453023541561186,
  'test': 0.46276087422160017},
 'user_crossover': {'train_eval': 7133, 'train_test': 5235, 'eval_test': 5350},
 'session_crossover': {'train_eval': 76, 'eval_test': 35}}

In [ ]:
with open('calculated_features/core_dict_30_files.json', 'w') as f:
    json.dump(core_dict, f, indent=4)

In [14]:
metadata_path = CORE_PATH + '/basic_descriptive_30_files.json'

In [15]:
with open(metadata_path) as f:
   meta = json.load(f)

In [16]:
df = pd.json_normalize(meta, max_level=0)

In [17]:
meta

{'n_events': {'train': 26950693, 'eval': 5775148, 'test': 5775149},
 'dates': {'train': {'min': '2021-10-19T08:40:37',
   'max': '2022-06-02T14:42:17'},
  'eval': {'min': '2022-06-02T14:42:18', 'max': '2022-07-09T05:43:02'},
  'test': {'min': '2022-07-09T05:43:04', 'max': '2022-08-14T05:13:27'}},
 'users': {'train': 81756, 'eval': 20111, 'test': 17376},
 'sessions': {'train': 466103, 'eval': 94549, 'test': 89998},
 'label_count': {'train': 0.381, 'eval': 0.445, 'test': 0.462},
 'user_crossover': {'train_eval': 7133, 'train_test': 5235, 'eval_test': 5350},
 'session_crossover': {'train_eval': 76, 'eval_test': 35}}

In [29]:
unravelled_dict = {}
cross_over_dict = {}
for k, v in meta.items():
    if 'train' in v or 'eval' in v or 'test' in v:
        unravelled_dict[k] = []
        unravelled_dict[k].append(v['train'])
        unravelled_dict[k].append(v['eval'])
        unravelled_dict[k].append(v['test'])

cross_over_dict['user_crossover'] = meta['user_crossover'].values()
cross_over_dict['session_crossover'] = list(meta['session_crossover'].values())+ [0]
unravelled_dict['date_min'] = [d['min'] for d in unravelled_dict['dates']]
unravelled_dict['date_max'] = [d['max'] for d in unravelled_dict['dates']]
del unravelled_dict['dates']


In [30]:
unravelled_dict
cross_over_dict

{'user_crossover': dict_values([7133, 5235, 5350]),
 'session_crossover': [76, 35, 0]}

In [43]:
df_unrav = pd.DataFrame(unravelled_dict)
df_cross = pd.DataFrame(cross_over_dict)
df_unrav = df_unrav,round(3)
df_cross = df_cross.round(3)

In [44]:
df_unrav = df_unrav[0]

In [45]:
df_unrav['dataset'] = ['train', 'eval', 'test']
df_cross['dataset'] = ['train eval', 'train test', 'eval test']


In [46]:
df_unrav.set_index('dataset', inplace=True)
df_cross.set_index('dataset', inplace=True)

In [48]:
print(df_unrav.round(3).to_latex())

\begin{tabular}{lrrrrll}
\toprule
 & n_events & users & sessions & label_count & date_min & date_max \\
dataset &  &  &  &  &  &  \\
\midrule
train & 26950693 & 81756 & 466103 & 0.381000 & 2021-10-19T08:40:37 & 2022-06-02T14:42:17 \\
eval & 5775148 & 20111 & 94549 & 0.445000 & 2022-06-02T14:42:18 & 2022-07-09T05:43:02 \\
test & 5775149 & 17376 & 89998 & 0.462000 & 2022-07-09T05:43:04 & 2022-08-14T05:13:27 \\
\bottomrule
\end{tabular}



In [42]:
print(df_cross.to_latex())

\begin{tabular}{lrr}
\toprule
 & user_crossover & session_crossover \\
dataset &  &  \\
\midrule
train eval & 7133 & 76 \\
train test & 5235 & 35 \\
eval test & 5350 & 0 \\
\bottomrule
\end{tabular}

